In [ ]:
import os
import json
from dotenv import load_dotenv
from googleapiclient.discovery import build

In [267]:
load_dotenv()
DEVELOPER_KEY = os.environ.get('yt_key')

YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'
yt = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

In [268]:
# Individuo le finestre temporali per le ricerche, con i rispettivi parametri
#windows = [
    #{"label": "approdo_maldini", "query": "Maldini Milan 2019 dirigenza", "after": "2019-05-20T00:00:00Z", "before": "2019-07-01T00:00:00Z"},
    #{"label": "scudetto", "query": "Milan campione 2022 Maldini", "after": "2022-05-15T00:00:00Z", "before": "2022-06-15T00:00:00Z"},
    #{"label": "licenziamento", "query": "Maldini licenziato Cardinale", "after": "2023-06-01T00:00:00Z", "before": "2023-06-30T00:00:00Z"},
    #{"label": "cardinale_out", "query": "#CardinaleOut Milan", "after": "2023-06-01T00:00:00Z", "before": "2023-12-31T00:00:00Z"}
#]

In [269]:
# 1. Recupera i canali
def get_channels(query, max_results):
    channels = dict()
    request = yt.search().list(
        part='snippet',
        q=query,
        type='channel',
        maxResults=max_results,
        relevanceLanguage='it'
    )
    try:
        response = request.execute()
        for item in response.get('items', []):
            channel_id = item['id']['channelId']
            channel_title = item['snippet']['title']
            channels[channel_id] = channel_title
        print(f"Canali trovati per la query '{query}': {len(channels)}")
    except Exception as e:
        print(f"Errore nel recupero canali: {e}")
    return channels

In [270]:
query = "AC Milan"
max_results = 50
channels = get_channels(query, max_results)
channels

Canali trovati per la query 'AC Milan': 50


{'UCKcx1uK38H4AOkmfv4ywlrg': 'AC Milan',
 'UC-Z7UsH1Qq3BEIXDoOHrQTQ': 'AC Milan Academy Junior Camp - Sporteventi',
 'UCfDu-gxOeG2nte4vcGBQB6w': 'Milan Actu',
 'UCkhRRSdaqpg3PNzwjLWcrQg': 'La Storia del Milan',
 'UCUzPh1oraBmxInFoZ0tmVng': 'La Taberna del Milan',
 'UCJv1AqHPPV0IXM5gB-L6-KA': 'Carlo Pellegatti',
 'UCYt5p_Qy9yC40ahQOUOAmXA': 'AC Milan Addicted',
 'UCYwM10PzkQHwkhr5CZ_uVzA': 'Lorenzo Lollo - I Milanisti Official',
 'UCH3oDvo6BGyXmQISU5cUWcA': 'Rinaldo Morelli',
 'UCa7LVlKqBQQgIIWc5bspGXA': 'AcmDevil',
 'UC4yO9MP6xvdtay7DXT1GrqA': 'Football Kush',
 'UCyA2Xjd0Pgo0XG2fwDNBMgQ': 'Alessandro Jacobone - AC Milan Live & Contents',
 'UCEq8kCs5hYGgmRpX7WJseLA': 'AC MILAN FEDEROSSONERA',
 'UCwncrOjASOSrPfSCBHe9Zfg': 'Vhs Dvd Ac Milan',
 'UCSqbYQ5MEyh06ep2ixJ2UiA': 'Ac Milan 2015',
 'UC5I4SUIhtFfmucafyv-xI1w': 'AC Milan Academy Kerala',
 'UCxNiO4bhMiTkisqaxoqS31A': 'AC Milan Academy Craiova',
 'UCrMHCcgSDdwX6JsKc7aLiDQ': 'Ac Milan Calcio',
 'UCwEPg_9gbLVDUbhZcc4hXNw': 'NOTIZIA AC MI

In [271]:
# 2. Recupera i video recenti da un canale
def get_recent_videos(channel_id, max_results=10):
    videos = []
    try:
        request = yt.search().list(
            part='snippet',
            channelId=channel_id,
            order='date',
            maxResults=max_results,
            type='video'
        )
        response = request.execute()
        for item in response.get('items', []):
            video_id = item['id']['videoId']
            videos.append(video_id)
        print(f"Video trovati per il canale {channel_id}: {len(videos)}")
    except Exception as e:
        print(f"Errore nel recupero video da {channel_id}: {e}")
    return videos

In [272]:
if channels:
    channel_id = list(channels.keys())[0]  
    videos = get_recent_videos(channel_id, max_results=10)

Video trovati per il canale UCKcx1uK38H4AOkmfv4ywlrg: 10


In [273]:
# 3. Recupera statistiche video
def get_video_stats(video_ids):
    stats = []
    for i in range(0, len(video_ids), 50):  
        try:
            request = yt.videos().list(
                part='statistics',
                id=",".join(video_ids[i:i+50])
            )
            response = request.execute()
            for item in response['items']:
                stats.append({
                    "videoId": item['id'],
                    "views": int(item['statistics'].get('viewCount', 0)),
                    "comments": int(item['statistics'].get('commentCount', 0)),
                })
            print(f"Statistiche recuperate per {len(stats)} video.")
        except Exception as e:
            print(f"Errore nel recupero statistiche video: {e}")
    return stats

In [274]:
if videos:
    stats = get_video_stats(videos)

Statistiche recuperate per 10 video.


In [275]:
# 4. Filtra canali per engagement medio, iscritti e numero minimo di video
def filter_channels_by_average_engagement(channels_dict, min_engagement=0.01, min_subscribers=10000, min_videos=20, max_videos_per_channel=20):
    filtered_channels = {}
    for channel_id, name in channels_dict.items():
        
        try:
            request = yt.channels().list(
                part="statistics",
                id=channel_id
            )
            response = request.execute()
            subscribers = int(response['items'][0]['statistics'].get('subscriberCount', 0))
            
            if subscribers < min_subscribers:
                print(f"[{name}] Non ha abbastanza iscritti (soglia: {min_subscribers}). Escluso.")
                continue
        except Exception as e:
            print(f"Errore nel recupero iscritti per il canale {name}: {e}")
            continue

        video_ids = get_recent_videos(channel_id, max_results=max_videos_per_channel)

        if len(video_ids) < min_videos:
            print(f"[{name}] Ha meno di {min_videos} video trovati. Escluso.")
            continue

        stats = get_video_stats(video_ids)

        engagements = []
        for s in stats:
            views = s['views']
            comments = s['comments']
            if views > 0:
                engagement = comments / views
                engagements.append(engagement)

        if engagements:
            avg_engagement = sum(engagements) / len(engagements)
            print(f"[{name}] Engagement medio: {avg_engagement:.4f}, Iscritti: {subscribers}, Video: {len(video_ids)}")

            if avg_engagement >= min_engagement:
                filtered_channels[channel_id] = {
                    "name": name,
                    "avg_engagement": round(avg_engagement, 4),
                    "subscribers": subscribers
                }
        else:
            print(f"[{name}] Nessun engagement valido trovato.")
    return filtered_channels

In [276]:
filtered_channels = filter_channels_by_average_engagement(channels)
filtered_channels

Video trovati per il canale UCKcx1uK38H4AOkmfv4ywlrg: 20
Statistiche recuperate per 20 video.
[AC Milan] Engagement medio: 0.0018, Iscritti: 2040000, Video: 20
Video trovati per il canale UC-Z7UsH1Qq3BEIXDoOHrQTQ: 20
Statistiche recuperate per 20 video.
[AC Milan Academy Junior Camp - Sporteventi] Engagement medio: 0.0002, Iscritti: 10300, Video: 20
Video trovati per il canale UCfDu-gxOeG2nte4vcGBQB6w: 20
Statistiche recuperate per 20 video.
[Milan Actu] Engagement medio: 0.0092, Iscritti: 25500, Video: 20
[La Storia del Milan] Non ha abbastanza iscritti (soglia: 10000). Escluso.
[La Taberna del Milan] Non ha abbastanza iscritti (soglia: 10000). Escluso.
Video trovati per il canale UCJv1AqHPPV0IXM5gB-L6-KA: 20
Statistiche recuperate per 20 video.
[Carlo Pellegatti] Engagement medio: 0.0317, Iscritti: 157000, Video: 20
[AC Milan Addicted] Non ha abbastanza iscritti (soglia: 10000). Escluso.
Video trovati per il canale UCYwM10PzkQHwkhr5CZ_uVzA: 20
Statistiche recuperate per 20 video.
[Lo

{'UCJv1AqHPPV0IXM5gB-L6-KA': {'name': 'Carlo Pellegatti',
  'avg_engagement': 0.0317,
  'subscribers': 157000},
 'UCYwM10PzkQHwkhr5CZ_uVzA': {'name': 'Lorenzo Lollo - I Milanisti Official',
  'avg_engagement': 0.0116,
  'subscribers': 30400},
 'UCH3oDvo6BGyXmQISU5cUWcA': {'name': 'Rinaldo Morelli',
  'avg_engagement': 0.0157,
  'subscribers': 35500},
 'UCa7LVlKqBQQgIIWc5bspGXA': {'name': 'AcmDevil',
  'avg_engagement': 0.0138,
  'subscribers': 10900},
 'UC4yO9MP6xvdtay7DXT1GrqA': {'name': 'Football Kush',
  'avg_engagement': 0.0143,
  'subscribers': 15200}}

In [277]:
# 5. Salva il risultato in JSON
def save_channels_to_json(channels_dict, filename='canali_engaged.json'):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(channels_dict, f, ensure_ascii=False, indent=4)

In [278]:
save_channels_to_json(filtered_channels)